In [ ]:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
#git clone https://github.com/Cyanogenoid/pytorch-vqa.git

import os, sys
sys.path.append(os.path.realpath('./pytorch-vqa'))

In [ ]:
# https://github.com/Cyanogenoid/pytorch-vqa/releases

#wget https://github.com/Cyanogenoid/pytorch-vqa/releases/download/v1.0/2017-08-04_00.55.19.pth  # 81Mb model

In [ ]:
import model

#log = torch.load('logs/2017-08-04_00:55:19.pth')
log = torch.load('./2017-08-04_00.55.19.pth')
tokens = len(log['vocab']['question']) + 1

net = torch.nn.DataParallel(model.Net(tokens))
net.load_state_dict(log['weights'])
net.to(device)

In [ ]:
log.keys()

In [ ]:
# Now let's try and answer a question on a single image...


In [ ]:
sys.path.append(os.path.realpath('./pytorch-resnet'))

In [ ]:
import resnet

import torchvision.transforms as transforms
from PIL import Image

def get_transform(target_size, central_fraction=1.0):
    return transforms.Compose([
        transforms.Scale(int(target_size / central_fraction)),
        transforms.CenterCrop(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

class ResNetFeatures(torch.nn.Module):
    def __init__(self):
        super(ResNetFeatures, self).__init__()
        self.model = resnet.resnet152(pretrained=True)

        def save_output(module, input, output):
            self.buffer = output
        self.model.layer4.register_forward_hook(save_output)

    def forward(self, x):
        self.model(x)
        return self.buffer

resnet_features = ResNetFeatures().to(device)  # Downloads 241Mb model when first run

def image_to_features(img_file):
    img = Image.open(img_file).convert('RGB')
    img_transformed = self.transform(img)
    return resnet_features(img_transformed)